<a href="https://colab.research.google.com/github/juhee3199/credit_score_predict/blob/main/baseline/Pycaret_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [ ]:
!pip install pycaret

     |████████████████████████████████| 266 kB 5.4 MB/s 
     |████████████████████████████████| 261 kB 26.0 MB/s 
     |████████████████████████████████| 271 kB 54.9 MB/s 
     |████████████████████████████████| 113 kB 40.9 MB/s 
     |████████████████████████████████| 6.8 MB 36.8 MB/s 
     |████████████████████████████████| 2.0 MB 35.9 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 16.9 MB 43 kB/s 
     |████████████████████████████████| 1.7 MB 28.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 167 kB 58.9 MB/s 
     |████████████████████████████████| 1.3 MB 37.0 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 675 kB 37.7 MB/s 
     |███████████████████

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from pycaret.classification import *
from sklearn.metrics import log_loss
import random

# Data Load & Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = "/content/drive/MyDrive/Colab Notebooks/dacon_credit_cart/credit_card_data/open"
train = pd.read_csv(PATH+'/train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = pd.read_csv(PATH+'/test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

submit = pd.read_csv(PATH+'/sample_submission.csv')

# Pycaret

In [ ]:
from pycaret.classification import *
from sklearn.metrics import log_loss

clf = setup(train, target = 'credit', train_size = 0.85)

,Description,Value
0,session_id,8068
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 19)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,13
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
#logloss 적용 및 predict_proba로 제출하기 위해 metric 추가
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7efe90855f80>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [ ]:
#svm, ridge는 predict_proba 미지원으로 제외
best5 = compare_models(fold = 5, sort = 'logloss', n_select = 5, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6986,0.7005,0.4268,0.6977,0.6221,0.2385,0.3236,0.7702,0.856
gbc,Gradient Boosting Classifier,0.6917,0.6481,0.4089,0.6703,0.6053,0.2070,0.3020,0.7983,8.220
lda,Linear Discriminant Analysis,0.6428,0.6039,0.3387,0.5369,0.5112,0.0153,0.0554,0.8647,0.286
nb,Naive Bayes,0.6423,0.6052,0.3349,0.5595,0.5039,0.0052,0.0387,0.8653,0.060
lr,Logistic Regression,0.6412,0.5681,0.3333,0.4112,0.5011,0.0000,0.0000,0.8764,1.100
rf,Random Forest Classifier,0.6998,0.7442,0.5390,0.6781,0.6833,0.3708,0.3772,1.0132,2.988
ada,Ada Boost Classifier,0.6903,0.6204,0.4041,0.6537,0.6005,0.1978,0.2993,1.0809,0.896
et,Extra Trees Classifier,0.6672,0.7056,0.5154,0.6466,0.6531,0.3113,0.3152,2.6439,3.268
knn,K Neighbors Classifier,0.6181,0.6533,0.4588,0.5934,0.6023,0.2098,0.2126,3.9748,0.756
dt,Decision Tree Classifier,0.6025,0.6336,0.4843,0.6116,0.6067,0.2478,0.2481,13.3125,0.244


In [ ]:
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'logloss')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6899,0.6529,0.4025,0.6250,0.5986,0.1929,0.2999,0.8226
1,0.6888,0.6709,0.4006,0.6151,0.5966,0.1915,0.2945,0.8174
2,0.6910,0.6555,0.4048,0.6200,0.6012,0.1999,0.3018,0.8162
3,0.6931,0.6556,0.4072,0.6301,0.6040,0.2035,0.3108,0.8158
4,0.6911,0.6475,0.4044,0.6245,0.6009,0.1978,0.3032,0.8190
Mean,0.6908,0.6565,0.4039,0.6229,0.6003,0.1971,0.3020,0.8182
SD,0.0015,0.0078,0.0022,0.0050,0.0025,0.0045,0.0053,0.0025


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6951,0,0.4089,0.6219,0.6068,0.2118,0.314,0


In [ ]:
#lda 와 nb는 losloss가 좋지만 accuracy, auc가 상대적으로 좋지 않아 커스텀모델 생성
custom_model = compare_models(fold = 5, sort = 'logloss', n_select = 6, include=['lightgbm','gbc','rf','ada', 'lda', 'nb'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6986,0.7005,0.4268,0.6977,0.6221,0.2385,0.3236,0.7702,0.792
gbc,Gradient Boosting Classifier,0.6917,0.6481,0.4089,0.6703,0.6053,0.2070,0.3020,0.7983,8.204
lda,Linear Discriminant Analysis,0.6428,0.6039,0.3387,0.5369,0.5112,0.0153,0.0554,0.8647,0.290
nb,Naive Bayes,0.6423,0.6052,0.3349,0.5595,0.5039,0.0052,0.0387,0.8653,0.086
rf,Random Forest Classifier,0.6998,0.7442,0.5390,0.6781,0.6833,0.3708,0.3772,1.0132,2.978
ada,Ada Boost Classifier,0.6903,0.6204,0.4041,0.6537,0.6005,0.1978,0.2993,1.0809,0.890


In [ ]:
blended_custom = blend_models(estimator_list = custom_model, fold = 5, optimize = 'logloss')
pred_holdout_custom = predict_model(blended_custom)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6910,0.7301,0.4081,0.6810,0.6042,0.2035,0.3003,0.7904
1,0.6939,0.7392,0.4106,0.6576,0.6077,0.2148,0.3094,0.7855
2,0.6943,0.7344,0.4119,0.6795,0.6089,0.2163,0.3106,0.7865
3,0.7000,0.7371,0.4186,0.7335,0.6163,0.2285,0.3328,0.7863
4,0.6954,0.7287,0.4132,0.6978,0.6102,0.2169,0.3152,0.7893
Mean,0.6949,0.7339,0.4125,0.6899,0.6095,0.2160,0.3137,0.7876
SD,0.0029,0.0040,0.0035,0.0253,0.0040,0.0079,0.0107,0.0019


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6992,0.7542,0.4171,0.7041,0.6153,0.2297,0.3252,0.7748


# Prediction

In [ ]:
#자동으로 상위 5개 선택한 모델
final_model = finalize_model(blended)

#Accurary, AUC, Logloss 셋다 상위 4개인 모델 
final_model_custom = finalize_model(blended_custom)

In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model_custom])
prections = prep_pipe.predict_proba(test)
prections

array([[0.12367595, 0.18558833, 0.69073572],
       [0.22207553, 0.20126551, 0.57665895],
       [0.13617258, 0.18497864, 0.67884878],
       ...,
       [0.10456021, 0.14766024, 0.74777955],
       [0.22081671, 0.20907472, 0.57010857],
       [0.14851869, 0.29123052, 0.56025079]])

In [ ]:
i, j, k = [], [], []
for row in prections:
  i.append(row[0])
  j.append(row[1])
  k.append(row[2])

len(i), len(j), len(k)

(10000, 10000, 10000)

In [ ]:
submit['0'] = i
submit['1'] = j
submit['2'] = k
submit.head(10)

,index,0,1,2
0,26457,0.123676,0.185588,0.690736
1,26458,0.222076,0.201266,0.576659
2,26459,0.136173,0.184979,0.678849
3,26460,0.127815,0.166802,0.705383
4,26461,0.149956,0.250547,0.599497
5,26462,0.138651,0.250649,0.610701
6,26463,0.350009,0.385505,0.264486
7,26464,0.144972,0.226139,0.628889
8,26465,0.125977,0.213399,0.660624
9,26466,0.128657,0.245017,0.626326


In [ ]:
#submit.to_csv(PATH+"\submisison.csv",index=False)

In [ ]:
submit.to_csv("submission.csv", index=False)